# Data Validation:

In [9]:
pip install from-root

Defaulting to user installation because normal site-packages is not writeable
  Using cached from_root-1.0.2-py3-none-any.whl (6.5 kB)

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install dill

Defaulting to user installation because normal site-packages is not writeable
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
import os
import sys
from distutils import dir_util

import pandas as pd

from sensor.constant.training_pipeline import SCHEMA_FILE_PATH
from sensor.entity.artifact_entity import (DataIngestionArtifact,DataValidationArtifact)
from sensor.entity.config_entity import DataValidationConfig
from sensor.exception import SensorException
from sensor.logger import logging
from sensor.utils.main_utils import read_yaml_file, write_yaml_file

In [22]:
SCHEMA_FILE_PATH

'config/schema.yaml'

In [38]:
DataIngestionArtifact.__annotations__

{'trained_file_path': str, 'test_file_path': str}

In [44]:
DataValidationArtifact.__annotations__

{'validation_status': bool,
 'valid_train_file_path': str,
 'valid_test_file_path': str,
 'invalid_train_file_path': str,
 'invalid_test_file_path': str,
 'drift_report_file_path': str}

## Steps in Data Validation: 
### 1. Validating number of columns
### 2. Validating data type of columns
### 2. Detecting data drift

## 1. Validating number of columns

In [81]:
schema_config = read_yaml_file("/Users/DEEPANSHU/Folder001/IRPC/APS-Fault-Detection/config/schema.yaml")

In [83]:
schema_config

{'columns': [{'class': 'category'},
  {'aa_000': 'int'},
  {'ac_000': 'int'},
  {'ad_000': 'int'},
  {'ae_000': 'int'},
  {'af_000': 'int'},
  {'ag_000': 'int'},
  {'ag_001': 'int'},
  {'ag_002': 'int'},
  {'ag_003': 'int'},
  {'ag_004': 'int'},
  {'ag_005': 'int'},
  {'ag_006': 'int'},
  {'ag_007': 'int'},
  {'ag_008': 'int'},
  {'ag_009': 'int'},
  {'ah_000': 'int'},
  {'ai_000': 'int'},
  {'aj_000': 'int'},
  {'ak_000': 'int'},
  {'al_000': 'int'},
  {'am_0': 'int'},
  {'an_000': 'int'},
  {'ao_000': 'int'},
  {'ap_000': 'int'},
  {'aq_000': 'int'},
  {'ar_000': 'int'},
  {'as_000': 'int'},
  {'at_000': 'int'},
  {'au_000': 'int'},
  {'av_000': 'int'},
  {'ax_000': 'int'},
  {'ay_000': 'int'},
  {'ay_001': 'int'},
  {'ay_002': 'int'},
  {'ay_003': 'int'},
  {'ay_004': 'int'},
  {'ay_005': 'int'},
  {'ay_006': 'int'},
  {'ay_007': 'int'},
  {'ay_008': 'int'},
  {'ay_009': 'int'},
  {'az_000': 'int'},
  {'az_001': 'int'},
  {'az_002': 'int'},
  {'az_003': 'int'},
  {'az_004': 'int'},


In [85]:
len(schema_config)

3

In [87]:
type(schema_config)

dict

In [89]:
schema_config.keys()

dict_keys(['columns', 'numerical_columns', 'drop_columns'])

In [86]:
len(schema_config["columns"])

164

In [16]:
SCHEMA_FILE_PATH

'config/schema.yaml'

In [90]:
train_df = pd.read_csv("/Users/DEEPANSHU/Folder001/IRPC/APS-Fault-Detection/artifact/11_22_2022_01_22_57/data_ingestion/ingested/train.csv")
test_df = pd.read_csv("/Users/DEEPANSHU/Folder001/IRPC/APS-Fault-Detection/artifact/11_22_2022_01_22_57/data_ingestion/ingested/test.csv")

In [91]:
train_df.head()

,class,aa_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,8,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,22.0,24.0,82.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0
1,neg,135064,5310.0,1794.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,neg,29698,420.0,398.0,0.0,0.0,0.0,0.0,0.0,0.0,...,257876.0,132972.0,232164.0,179282.0,152876.0,107718.0,219230.0,5332.0,0.0,0.0
3,pos,767028,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,8652934.0,4023780.0,10336264.0,8726276.0,2702378.0,606250.0,297838.0,1690.0,0.0,0.0
4,neg,49428,94.0,92.0,0.0,0.0,0.0,0.0,0.0,0.0,...,152416.0,73996.0,167820.0,161014.0,322384.0,1199314.0,13592.0,0.0,0.0,0.0


In [94]:
def validate_number_of_columns(dataframe:pd.DataFrame)->bool:
    number_of_columns = len(schema_config["columns"])
    if len(dataframe.columns)==number_of_columns:
        return True
    return False

In [96]:
validate_number_of_columns(train_df)

True

In [97]:
validate_number_of_columns(test_df)

True

## 2. Validating datatype of columns

In [98]:
numerical_columns = schema_config["numerical_columns"]
numerical_columns

['aa_000',
 'ac_000',
 'ad_000',
 'ae_000',
 'af_000',
 'ag_000',
 'ag_001',
 'ag_002',
 'ag_003',
 'ag_004',
 'ag_005',
 'ag_006',
 'ag_007',
 'ag_008',
 'ag_009',
 'ah_000',
 'ai_000',
 'aj_000',
 'ak_000',
 'al_000',
 'am_0',
 'an_000',
 'ao_000',
 'ap_000',
 'aq_000',
 'ar_000',
 'as_000',
 'at_000',
 'au_000',
 'av_000',
 'ax_000',
 'ay_000',
 'ay_001',
 'ay_002',
 'ay_003',
 'ay_004',
 'ay_005',
 'ay_006',
 'ay_007',
 'ay_008',
 'ay_009',
 'az_000',
 'az_001',
 'az_002',
 'az_003',
 'az_004',
 'az_005',
 'az_006',
 'az_007',
 'az_008',
 'az_009',
 'ba_000',
 'ba_001',
 'ba_002',
 'ba_003',
 'ba_004',
 'ba_005',
 'ba_006',
 'ba_007',
 'ba_008',
 'ba_009',
 'bb_000',
 'bc_000',
 'bd_000',
 'be_000',
 'bf_000',
 'bg_000',
 'bh_000',
 'bi_000',
 'bj_000',
 'bk_000',
 'bl_000',
 'bm_000',
 'bs_000',
 'bt_000',
 'bu_000',
 'bv_000',
 'bx_000',
 'by_000',
 'bz_000',
 'ca_000',
 'cb_000',
 'cc_000',
 'cd_000',
 'ce_000',
 'cf_000',
 'cg_000',
 'ch_000',
 'ci_000',
 'cj_000',
 'ck_000',
 

In [99]:
len(numerical_columns)

163

In [100]:
train_df.columns

Index(['class', 'aa_000', 'ac_000', 'ad_000', 'ae_000', 'af_000', 'ag_000',
       'ag_001', 'ag_002', 'ag_003',
       ...
       'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008',
       'ee_009', 'ef_000', 'eg_000'],
      dtype='object', length=164)

In [103]:
for i in numerical_columns:
    if i not in train_df.columns:
        print("Error")

In [104]:
def is_numerical_column_exist(dataframe:pd.DataFrame)->bool:

    numerical_columns = schema_config["numerical_columns"]
    dataframe_columns = dataframe.columns
    numerical_column_present = True
    missing_numerical_columns = []
    for num_column in numerical_columns:
        if num_column not in dataframe_columns:
            numerical_column_present=False
            missing_numerical_columns.append(num_column)

    return numerical_column_present

In [105]:
is_numerical_column_exist(train_df)

True

In [107]:
# dropping numerical columns deliberately to check the function
df2 = train_df.copy()
df2.drop("ac_000",axis = 1, inplace=True)

In [108]:
is_numerical_column_exist(df2)

False

## Detecting Data drift

In [109]:
train_df

,class,aa_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,8,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,22.0,24.0,82.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0
1,neg,135064,5310.0,1794.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,neg,29698,420.0,398.0,0.0,0.0,0.0,0.0,0.0,0.0,...,257876.0,132972.0,232164.0,179282.0,152876.0,107718.0,219230.0,5332.0,0.0,0.0
3,pos,767028,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,8652934.0,4023780.0,10336264.0,8726276.0,2702378.0,606250.0,297838.0,1690.0,0.0,0.0
4,neg,49428,94.0,92.0,0.0,0.0,0.0,0.0,0.0,0.0,...,152416.0,73996.0,167820.0,161014.0,322384.0,1199314.0,13592.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,neg,60238,1162.0,958.0,0.0,0.0,0.0,0.0,0.0,0.0,...,845020.0,262370.0,571000.0,522624.0,267102.0,125418.0,47588.0,538.0,0.0,0.0
15996,neg,16510,126.0,124.0,0.0,0.0,0.0,0.0,0.0,0.0,...,126008.0,50202.0,95742.0,98032.0,95706.0,243508.0,3798.0,0.0,0.0,0.0
15997,neg,66256,644.0,NaN,0.0,0.0,0.0,0.0,0.0,28950.0,...,418638.0,157426.0,283168.0,341816.0,680494.0,945028.0,26226.0,0.0,2.0,4.0
15998,neg,38886,126.0,124.0,0.0,0.0,0.0,0.0,0.0,0.0,...,389614.0,212934.0,401256.0,332584.0,189440.0,95130.0,114798.0,6246.0,0.0,0.0


In [110]:
test_df

,class,aa_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,57140,2.130706e+09,1048.0,0.0,0.0,0.0,0.0,0.0,0.0,...,502516.0,240194.0,482820.0,417904.0,374770.0,224304.0,290198.0,7242.0,0.0,0.0
1,neg,126566,9.140000e+02,226.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1100358.0,534282.0,1111740.0,874320.0,714000.0,473336.0,763430.0,46694.0,0.0,0.0
2,neg,28202,2.660000e+02,256.0,0.0,0.0,0.0,0.0,0.0,0.0,...,279646.0,157246.0,431052.0,328130.0,112122.0,23792.0,6996.0,28.0,0.0,0.0
3,neg,31082,2.000000e+02,190.0,0.0,0.0,0.0,0.0,0.0,0.0,...,319140.0,160646.0,319082.0,324442.0,200886.0,89398.0,29118.0,0.0,0.0,0.0
4,neg,120744,0.000000e+00,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,966388.0,569148.0,2113720.0,2094512.0,43860.0,1350.0,132.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3996,neg,42364,5.900000e+02,540.0,0.0,0.0,0.0,0.0,0.0,0.0,...,411944.0,213500.0,420686.0,407552.0,211484.0,109036.0,144224.0,6024.0,0.0,0.0
3997,neg,1282,2.130706e+09,106.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7598.0,1836.0,5386.0,3164.0,8124.0,26748.0,0.0,0.0,0.0,0.0
3998,neg,31124,2.130706e+09,640.0,0.0,0.0,0.0,0.0,0.0,0.0,...,349920.0,178456.0,453852.0,434946.0,23498.0,2734.0,1146.0,0.0,0.0,0.0
3999,neg,5640,1.340000e+02,118.0,0.0,0.0,0.0,0.0,0.0,0.0,...,43110.0,11676.0,20614.0,30760.0,19210.0,19826.0,54060.0,5716.0,0.0,0.0


In [112]:
from scipy.stats import ks_2samp

In [116]:
train_df.iloc[:,1:]

,aa_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,ag_004,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,8,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1538.0,...,22.0,24.0,82.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0
1,135064,5310.0,1794.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,29698,420.0,398.0,0.0,0.0,0.0,0.0,0.0,0.0,334.0,...,257876.0,132972.0,232164.0,179282.0,152876.0,107718.0,219230.0,5332.0,0.0,0.0
3,767028,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,8770.0,...,8652934.0,4023780.0,10336264.0,8726276.0,2702378.0,606250.0,297838.0,1690.0,0.0,0.0
4,49428,94.0,92.0,0.0,0.0,0.0,0.0,0.0,0.0,285098.0,...,152416.0,73996.0,167820.0,161014.0,322384.0,1199314.0,13592.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,60238,1162.0,958.0,0.0,0.0,0.0,0.0,0.0,0.0,83582.0,...,845020.0,262370.0,571000.0,522624.0,267102.0,125418.0,47588.0,538.0,0.0,0.0
15996,16510,126.0,124.0,0.0,0.0,0.0,0.0,0.0,0.0,113236.0,...,126008.0,50202.0,95742.0,98032.0,95706.0,243508.0,3798.0,0.0,0.0,0.0
15997,66256,644.0,NaN,0.0,0.0,0.0,0.0,0.0,28950.0,878464.0,...,418638.0,157426.0,283168.0,341816.0,680494.0,945028.0,26226.0,0.0,2.0,4.0
15998,38886,126.0,124.0,0.0,0.0,0.0,0.0,0.0,0.0,144.0,...,389614.0,212934.0,401256.0,332584.0,189440.0,95130.0,114798.0,6246.0,0.0,0.0


In [119]:
columns = train_df.columns

In [120]:
columns

Index(['class', 'aa_000', 'ac_000', 'ad_000', 'ae_000', 'af_000', 'ag_000',
       'ag_001', 'ag_002', 'ag_003',
       ...
       'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008',
       'ee_009', 'ef_000', 'eg_000'],
      dtype='object', length=164)

In [121]:
train_df[columns[0]]

0        neg
1        neg
2        neg
3        pos
4        neg
        ... 
15995    neg
15996    neg
15997    neg
15998    neg
15999    neg
Name: class, Length: 16000, dtype: object

In [123]:
for i in columns:
    print(ks_2samp(train_df[i], test_df[i]))

KstestResult(statistic=0.0007536865783553948, pvalue=1.0)
KstestResult(statistic=0.009323965883529128, pvalue=0.9410926404905947)
KstestResult(statistic=0.014956229692576928, pvalue=0.4666665269460183)
KstestResult(statistic=0.01571866408397904, pvalue=0.4035151787899641)
KstestResult(statistic=0.006077121344663761, pvalue=0.9997476525982335)
KstestResult(statistic=0.006138059235191173, pvalue=0.9996944698488477)
KstestResult(statistic=0.0016901868282929167, pvalue=1.0)
KstestResult(statistic=0.002440749187703095, pvalue=1.0)
KstestResult(statistic=0.004233754061484629, pvalue=0.9999999921393242)
KstestResult(statistic=0.021381826418395367, pvalue=0.10558571481456991)
KstestResult(statistic=0.014510091227193156, pvalue=0.505837630438867)
KstestResult(statistic=0.011979083354161424, pvalue=0.743136796721573)
KstestResult(statistic=0.018725287428142945, pvalue=0.20903060791721806)
KstestResult(statistic=0.02025673269182704, pvalue=0.14259543852717427)
KstestResult(statistic=0.01975140589

KstestResult(statistic=0.011775259310172426, pvalue=0.7618139687233745)
KstestResult(statistic=0.011778446013496646, pvalue=0.7615244091494249)
KstestResult(statistic=0.016462478130467417, pvalue=0.3470947984351892)
KstestResult(statistic=0.016162225068732772, pvalue=0.3692243802783932)
KstestResult(statistic=0.007402102599350147, pvalue=0.994232207459485)
KstestResult(statistic=0.009301862034491348, pvalue=0.9421660549546861)
KstestResult(statistic=0.0069572919270182165, pvalue=0.9975278360137592)
KstestResult(statistic=0.011081104723819069, pvalue=0.8224616482586455)
KstestResult(statistic=0.005698497250687273, pvalue=0.9999334407785435)
KstestResult(statistic=0.005510372406898334, pvalue=0.9999691532496195)
KstestResult(statistic=0.01402252561859535, pvalue=0.5502004624889628)
KstestResult(statistic=0.013592226943264119, pvalue=0.5903981557771982)
KstestResult(statistic=0.012395776055985963, pvalue=0.7041646483947299)
KstestResult(statistic=0.01873940889777559, pvalue=0.208324210912

In [124]:
def detect_dataset_drift(base_df,current_df,threshold=0.05)->bool:
    
    status=True
    report ={}
    for column in base_df.columns:
        d1 = base_df[column]
        d2  = current_df[column]
        is_same_dist = ks_2samp(d1,d2)
        if threshold<=is_same_dist.pvalue:
            is_found=False
        else:
            is_found = True 
            status=False
        report.update({column:{
            "p_value":float(is_same_dist.pvalue),
            "drift_status":is_found

            }})

    return report

In [125]:
detect_dataset_drift(train_df, test_df)

{'class': {'p_value': 1.0, 'drift_status': False},
 'aa_000': {'p_value': 0.9410926404905947, 'drift_status': False},
 'ac_000': {'p_value': 0.4666665269460183, 'drift_status': False},
 'ad_000': {'p_value': 0.4035151787899641, 'drift_status': False},
 'ae_000': {'p_value': 0.9997476525982335, 'drift_status': False},
 'af_000': {'p_value': 0.9996944698488477, 'drift_status': False},
 'ag_000': {'p_value': 1.0, 'drift_status': False},
 'ag_001': {'p_value': 1.0, 'drift_status': False},
 'ag_002': {'p_value': 0.9999999921393242, 'drift_status': False},
 'ag_003': {'p_value': 0.10558571481456991, 'drift_status': False},
 'ag_004': {'p_value': 0.505837630438867, 'drift_status': False},
 'ag_005': {'p_value': 0.743136796721573, 'drift_status': False},
 'ag_006': {'p_value': 0.20903060791721806, 'drift_status': False},
 'ag_007': {'p_value': 0.14259543852717427, 'drift_status': False},
 'ag_008': {'p_value': 0.16232824658953982, 'drift_status': False},
 'ag_009': {'p_value': 0.26624010751996